In [0]:
from google.colab import drive
import os

MOUNT_DIR = '/content/drive'
drive.mount(MOUNT_DIR, force_remount=True)

In [0]:
ROOT_DIR = '/content/drive/My Drive/Colab Notebooks/EIP4'
MODELS_DIR = os.path.join(ROOT_DIR, 'saved_models')

In [0]:
!rm -rf /content/resized

In [0]:

# mount gdrive and unzip data
!unzip -q "/content/drive/My Drive/Colab Notebooks/EIP4/Assignment5_Dataset/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import keras
from keras.applications import VGG16, ResNet50, InceptionV3
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

In [0]:
# load annotations

df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head(), df.shape



In [0]:
# one hot encoding of labels

one_hot_df = pd.concat(
    [
        df[["image_path"]],
        pd.get_dummies(df.gender, prefix="gender"),
        pd.get_dummies(df.imagequality, prefix="imagequality"),
        pd.get_dummies(df.age, prefix="age"),
        pd.get_dummies(df.weight, prefix="weight"),
        pd.get_dummies(df.carryingbag, prefix="carryingbag"),
        pd.get_dummies(df.footwear, prefix="footwear"),
        pd.get_dummies(df.emotion, prefix="emotion"),
        pd.get_dummies(df.bodypose, prefix="bodypose"),
    ], 
    axis = 1
)

one_hot_df.head().T

In [0]:
# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

In [0]:
class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation = None): #img_aug=False):
        # self.img_aug = img_aug
        self.augmentation = augmentation
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])

        # if self.img_aug:
        #   eraser = get_random_eraser()
        #   aug_image_stack = np.stack([eraser(img) for img in image])
        if self.augmentation is not None:
          image = self.augmentation.flow(image, shuffle=False).next()

        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }

        # if self.img_aug:
        #   return aug_image_stack, target

        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=ImageDataGenerator(horizontal_flip=True))#img_aug=True)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False) #, img_aug=False)


In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
backbone = InceptionV3(
    weights=None, 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)
# neck = BatchNormalization()(neck) ####

def build_tower(in_layer):
    # neck = Dropout(0.2)(in_layer)
    # neck = Dense(128, activation="relu")(neck)
    # neck = BatchNormalization()(neck) ####
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    # neck = BatchNormalization()(neck) ####
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = True

In [0]:
losses = {
	"gender_output": "binary_crossentropy",
	"image_quality_output": "categorical_crossentropy",
	"age_output": "categorical_crossentropy",
	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output":"categorical_crossentropy",
  "emotion_output":"categorical_crossentropy"

}
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = SGD(lr=1e-5, momentum=0.9)
model.compile(
    optimizer=opt,
    # loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    loss = losses,
    metrics=["accuracy"]
)

In [0]:
model.summary()

In [0]:
history = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=20,
    callbacks=[
            ModelCheckpoint(
                os.path.join(MODELS_DIR, 'model_{epoch:03d}.hdf5'),
                save_best_only=True,
                verbose=1,
            )],
    verbose=1
)

In [0]:
model = load_model('/content/drive/My Drive/Colab Notebooks/EIP4/saved_models/model_020.hdf5')
K.set_value(model.optimizer.lr, 1e-5)

In [0]:
history = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=20,
    callbacks=[
            ModelCheckpoint(
                os.path.join(MODELS_DIR, 'model2_{epoch:02d}.hdf5'),
                save_best_only=True,
                verbose=1,
            )],
    verbose=1
)

In [0]:
model = load_model('/content/drive/My Drive/Colab Notebooks/EIP4/saved_models/model2_20.hdf5')

In [0]:
K.set_value(model.optimizer.lr, 6e-4)
history = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    callbacks=[
            ModelCheckpoint(
                os.path.join(MODELS_DIR, 'e64model_{epoch:02d}.hdf5'),
                save_best_only=True,
                verbose=1,
            )],
    verbose=1
)

In [0]:
model = load_model('/content/drive/My Drive/Colab Notebooks/EIP4/saved_models/e64model_09.hdf5')
K.set_value(model.optimizer.lr, 6e-4)

In [0]:
history = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=20,
    callbacks=[
            ModelCheckpoint(
                os.path.join(MODELS_DIR, 'e64model2_{epoch:02d}.hdf5'),
                save_best_only=True,
                verbose=1,
            )],
    verbose=1
)

In [0]:
model = load_model('/content/drive/My Drive/Colab Notebooks/EIP4/saved_models/e64model_09.hdf5')
K.set_value(model.optimizer.lr, 1e-4)

In [0]:
K.set_value(model.optimizer.lr, 2e-5)

In [0]:
MODELS_DIR = os.path.join(ROOT_DIR, 'inception_models') 
history = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=30,
    callbacks=[
            ModelCheckpoint(
                os.path.join(MODELS_DIR, 'i2e5model_{epoch:03d}.hdf5'),
                save_best_only=True,
                verbose=1,
            )],
    verbose=1
)

In [0]:
model = load_model('/content/drive/My Drive/Colab Notebooks/EIP4/inception_models/i2e5model_006.hdf5')

In [0]:
K.set_value(model.optimizer.lr, 1e-5)
MODELS_DIR = os.path.join(ROOT_DIR, 'inception_models') 
history = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    callbacks=[
            ModelCheckpoint(
                os.path.join(MODELS_DIR, 'i1e5model_{epoch:03d}.hdf5'),
                save_best_only=True,
                verbose=1,
            )],
    verbose=1
)